# Pytorch 自定义算子

PyTorch 提供了大量适用于张量的运算符库（例如 `torch.add`、`torch.sum` 等）。但是，您可能希望将新的自定义运算引入 PyTorch，并使其与 `torch.compile`、`autograd` 和 `torch.vmap` 等子系统配合使用。为此，您必须通过 Python [torch.library](https://pytorch.org/docs/stable/library.html) 文档或 C++ `TORCH_LIBRARY` API 将自定义算子注册到 PyTorch。

## 使用 Python 编写自定义运算符

请参阅[自定义 Python 运算符](https://pytorch.org/tutorials/advanced/python_custom_ops.html#python-custom-ops-tutorial)。

如果出现以下情况，您可能希望使用 Python（而不是 C++）编写自定义算子：
+ 您有一个 Python 函数，希望 PyTorch 将其视为不透明可调用函数，尤其是对于 `torch.compile` 和 `torch.export`。
+ 您有一些与 C++/CUDA 内核的 Python 绑定，并希望它们与 PyTorch 子系统（如 `torch.compile` 或 `torch.autograd`）组合
+ 您正在使用 Python（而不是仅使用 C++ 的环境，如 AOTInductor）。

## 将自定义 C++/CUDA 代码与 PyTorch 集成

请参阅[自定义 C++ 和 CUDA 运算符](https://pytorch.org/tutorials/advanced/cpp_custom_ops.html#cpp-custom-ops-tutorial)。

如果满足以下条件，您可能希望使用 C++（而不是 Python）编写自定义算子：
+ 您有自定义 C++/CUDA 代码。
+ 您计划将此代码与 AOTInductor 一起使用以进行无 Python 推理。

## 自定义运算符手册

有关教程和此页面中未涵盖的信息，请参阅[自定义算子手册](https://docs.google.com/document/d/1_W62p8WJOQQUzPsJYa7s701JXt0qf2OfLub2sbkHOaU)（我们正在努力将信息移至我们的文档网站）。我们建议您首先阅读上述教程之一，然后使用自定义算子手册作为参考；它不打算从头到脚阅读。

### 我应该何时创建自定义运算符？

**如果您的操作可以表示为内置 PyTorch 运算符的组合，那么请将其编写为 Python 函数并调用它，而不是创建自定义运算符。**如果您要调用 PyTorch 无法理解的某些库（例如自定义 C/C++ 代码、自定义 CUDA 内核或 Python 绑定到 C/C++/CUDA 扩展），请使用算子注册 API 创建自定义运算符。

### 为什么要创建自定义运算符？

可以通过抓取 Tensor 的数据指针并将其传递给 pybind 内核来使用 C/C++/CUDA 内核。但是，这种方法无法与 PyTorch 子系统（如 `autograd`、`torch.compile`、`vmap` 等）组合使用。**为了使操作能够与 PyTorch 子系统组合使用，必须通过算子注册 API 进行注册。**